This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

KRfk96yoWvruWZ-LjPb


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data')

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

data = r.json() # contains one object with key 'dataset_data'

data['dataset_data'].keys() # ['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order']
data['dataset_data']['column_names']


['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

r = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31')

In [6]:
# 2. Convert the returned JSON object into a Python dictionary.

data = r.json()
data = data['dataset_data']

In [7]:
cleaned_data = [{
    'Date': entry[0], 
    'Open': entry[1],
    'High': entry[2],
    'Low': entry[3],
    'Close': entry[4],
    'Change': entry[5],
    'Traded Volume': entry[6],
    'Turnover': entry[7],
    'Last Price of the Day': entry[8],
    'Daily Traded Units': entry[9],
    'Daily Turnover': entry[10]
} for entry in data['data']]


In [8]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.

highest_opening_price = max([entry['Open'] for entry in cleaned_data if entry['Open']])
lowest_opening_price = min([entry['Open'] for entry in cleaned_data if entry['Open']])

highest_opening_price, lowest_opening_price

(53.11, 34.0)

In [9]:
# 4. What was the largest change in any one day (based on High and Low price)?

largest_change_in_one_day = max([entry['High'] - entry['Low'] for entry in cleaned_data])
largest_change_in_one_day

2.8100000000000023

In [10]:
# 5. What was the largest change between any two days (based on Closing Price)?
# (Assuming this is the difference between any two days here, not necessarily consecutive)

largest_change_between_two_days = max([entry['Close'] for entry in cleaned_data if entry['Close']]) - min([entry['Close'] for entry in cleaned_data if entry['Close']])
largest_change_between_two_days

19.03

In [11]:
# 6. What was the average daily trading volume during this year?

traded_volumes = [entry['Traded Volume'] for entry in cleaned_data if entry['Traded Volume']]
average_daily_traded_volume = avg = sum(traded_volumes)/len(traded_volumes)
average_daily_traded_volume

89124.33725490196

In [12]:
# 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

sorted_traded_volumes = sorted(traded_volumes)
mid = len(sorted_traded_volumes) // 2
median_traded_volume = (sorted_traded_volumes[mid] + sorted_traded_volumes[~mid]) / 2
median_traded_volume

76286.0